<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.1.1
# *Data Wrangling and Munging with Pandas*

## Part 1: Wrangling Data

The term "data wrangling" is analogous to capturing wild horses and getting them into a fenced area; the horses are data and the fencing is your computer. The more common data wrangling tasks include:

- reading flat files
- reading Excel files
- downloading from web pages
  - csv
  - html
  - json

In [193]:
import numpy as np
import pandas as pd

*It is good practice to display the library version numbers for future reference:*

In [194]:
print('Numpy: ', np.__version__)
print('Pandas: ', pd.__version__)

Numpy:  1.26.0
Pandas:  2.1.1


### CSV Files

Below are three attempts to load the file "bikeshare.csv" into a DataFrame named `bikes`. Why are they wrong?

In [195]:
# wrong:
bikes = pd.read_table('bikeshare.csv', header = None)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('bikeshare.csv', header = 1)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('bikeshare.csv', header = 0)
print(bikes.head())

FileNotFoundError: [Errno 2] No such file or directory: 'bikeshare.csv'

ANSWER:

Load the file "bikeshare.csv" into a DataFrame named `bikes`, and confirm that it was loaded properly:

In [ ]:
#ANSWER:
bikes = pd.read_csv('../../DATA/bikeshare.csv')
print(bikes.head())

Note that we could have used `read.csv()` above. When is `read_table()` necessary?

?:
ANSWER: When `sep` is not the comma character, or we need fine control that `read.csv()` does not provide.

Flat files can be full of surprises. Here are some issues to watch out for:

- separator character is something other than the comma
  - ";", "|", and tab are popular
- newline character is something other than what the O/S expects
  - Tip: Don't hard-code the character codes for carriage returns, linefeeds, etc. Use Python's built-in representation instead (e.g. Python translates "\n" to the newline character and "\t" to the tab character on any O/S).
- truncated lines
  - if there are empty fields at the end of a line it is possible that their separators will be missing, resulting in a "jagged" file
- embedded commas or quotes
  - a free-text field containing embedded commas may split into separate fields on input
  - a free-text field containing embedded quotes may not parse correctly
- unescaped characters
  - the "\" character indicates a control code to Python, which will break the I/O
    - e.g. the substring "\u0123" will be interpreted as Unicode(0123) -- which may not be what the file creator intended
  - these may need to be fixed by loading whole strings and then parsing into a new data frame
  
Tip: Most issues can be dealt with by correctly specifying the parameters of the function you use to load the file. Read the doco before reading the data!

### Reading Excel Files

In [ ]:
from pandas import ExcelFile  # Nb. Need to install xlrd from conda (it does not automatically install with pandas)

In [ ]:
df = pd.read_excel('../../DATA/Iris.xls', sheet_name = 'Data')
df

It is usually better to load data correctly than to meddle with the source file or load it 'warts and all' and then try to parse it in code. The Pandas functions for reading files have parameters that provide the control we need. For example, we could make multiple calls to `read_excel()`, using combinations of the `header`, `usecols`, `skiprows`, `nrows`, and `skipfooter` parameters to load one table at a time from a spreadsheet with multiple tables.

Load the above file without the unwanted columns:

In [ ]:
#ANSWER

df = pd.read_excel('../../DATA/Iris.xls', sheet_name = 'Data')
df.head()

### Importing Data Directly from the Web

We usually want to store a local copy of a data file that we download from the Web, but when data retention is not a priority it is convenient to download the data directly into our running Python environment.

#### Importing Text Files from the Web

The web is the 'wild west' of data formats. However, we can usually expect good behaviour from files that are automatically generated by a service, such as the earthquake report:

In [ ]:
df = pd.read_csv('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_hour.csv')
df.head()

#### Importing HTML Files from the Web

Working with unstructured HTML files relies heavily on library functions. This one, however, is well-structured:

In [ ]:
url = 'https://www.ccra.com/airport-codes/'

df = pd.read_html(url)
df

In [ ]:
df[0]

#### Importing XML Files from the Web

XML files are semi-structured, but you're at the mercy of the file creator. If every record has the same format it will be much easier, but practical applications often require a lot of custom code. Here are a few examples: https://pandas.pydata.org/docs/user_guide/io.html#io-read-xml

#### Importing JSON Files from the Web

Like XML, JSON files are semi-structured and may require work to capture the schema into a dataframe. Here is a simple example:

In [ ]:
url = 'https://microsoftedge.github.io/Demos/json-dummy-data/64KB.json'

# Load the first sheet of the JSON file into a data frame
df = pd.read_json(url, orient = 'columns')
df.head()

## Part 2: Data Munging

Data munging is manipulating data to get it into a form that we can start running analyses on (which usually means getting the data into a DataFrame). Before we get to this stage, we may need to remove headers or footers, transpose columns to rows, split wide data tables into long ones, and so on. (Nb. Excel files can be particularly troublesome, because users can format their data in mixed, complex shapes.) Essentially, we need to follow Hadley Wickham's guidelines for tidy datasets (http://vita.had.co.nz/papers/tidy-data.html):

The end goal of the cleaning data process:

- each variable should be in one column
- each observation should comprise one row
- each type of observational unit should form one table
- include key columns for linking multiple tables
- the top row contains (sensible) variable names
- in general, save data as one file per table


### Dataset Morphology

Once we have our dataset in a DataFrame (or Series, if our data is only 1-dimensional), we can start examining its size and content.

How many rows and columns are in `bikes`?

In [ ]:
#ANSWER
bikes.info()

What are the column names in `bikes`?

In [ ]:
#ANSWER
bikes.columns.values

What are the data types of these columns?

In [ ]:
#ANSWER
bikes.dtypes

What is the (row) index for this DataFrame?

In [ ]:
#ANSWER
bikes.index

https://www.dataquest.io/blog/python-json-tutorial/

## Slicing and Dicing

It is often preferable to refer to DataFrame columns by name, but there is more than one way to do this.
Do `bikes['season']` and `bikes[['season']]` give the same object? Demonstrate:

In [ ]:
#ANSWER
bikes['season']  # looks up by key and returns a series object 

In [ ]:
bikes[['season']]  # looks up all matching and returns a dataframe object 

How would we use object notation to show the first 4 rows of `atemp`?

In [ ]:
#ANSWER
bikes['atemp'][:4]

In [ ]:
bikes.head()

Algorithms that loop over multiple columns often access DataFrame columns by index. However, none of the following work (try them out by uncommenting / removing the "#E: " ):

In [ ]:
# bikes[[0]]
# bikes[0]
# bikes[0,0]
# bikes[[0,0]]

What is the correct way to access the 1st row of the DataFrame by its index?

In [ ]:
#ANSWER
bikes.iloc[0]

What is the correct way to access the 2nd column of the DataFrame by its index?

In [ ]:
#ANSWER
bikes.iloc[:,1]

In [ ]:
bikes.head()

## Handling Missing Values

What is the Pandas `isnull` function for?

?
ANSWER:

We can apply `isnull` to the `bikes` DataFrame to show the result for every element:

In [ ]:
bikes.isnull().head()

However, we usually start at a higher level. How many nulls are in `bikes` altogether?

In [ ]:
#ANSWER
bikes.isnull().values.sum()

If this result were nonzero we would next want to find out which columns contained nulls. How can this be done in one line of code?

In [ ]:
#ANSWER
bikes.isnull().any(axis=0)

What is the Numpy object `nan` used for? (Write a descriptive answer.)

?
ANSWER: NaN stands for Not a Number and represents a number that is undefined or unprepresentable, such as 0/0 or sqrt of a negative number. 

Write (and verify) a function that performs scalar division with built-in handling of the edge case (i.e. return a value instead of just trapping the error):

In [ ]:
#ANSWER

def scalar_div(a, b):
    if b == 0:
        return "NaN"
    else:
        return a/b

print(scalar_div(4, 0))
print(scalar_div(4, 3))

Apply the Pandas `isna` function to the following data objects:

In [ ]:
x = 2.3
y = np.nan
print(x, y)

In [ ]:
#ANSWER
x = 2.3
y = np.nan
nums = pd.Series([x, y])
missing_values = nums.isna()
print(missing_values)

In [ ]:
array = np.array([[1, np.nan, 3], [4, 5, np.nan]])
print(array)

In [ ]:
#ANSWER
missing_values = pd.isna(array)
print(missing_values)

How is the pandas I/O parameter `na_values` used?

? ANSWER: It allows additional values to be recognised as NaN when reading and converting a file into a dataframe. 

## Data Profiling

### Counts

When there are categorical variables in a dataset we will want to know how many possible values there are in each column. (Nb. If the dataset is a sample of a larger one, our sample may not capture all possible values of every categorical.)

How many (different) seasons are in `bikes`?

In [ ]:
#ANSWER
bikes['season'].value_counts()

In [ ]:
bikes.head()

### Ranges

Print the range of the `instant`, `dteday`, and `windspeed` columns:

In [ ]:
bikes['dteday'].max()

In [ ]:
#ANSWER
bikes['dteday'] = pd.to_datetime(bikes['dteday'])
print("Range for instant " + str(bikes['instant'].max() - bikes['instant'].min()))
print("Range for dteday " + str((bikes['dteday'].max() - bikes['dteday'].min()).days) + " days")
print("Range for windspeed " + str(bikes['windspeed'].max() - bikes['windspeed'].min()))

Compute and print the overall minimum and maximum of the numeric data columns:

In [ ]:
bikes_min, bikes_max = (bikes.min(), bikes.max())
bikes_min, bikes_max

### Quantiles

Pandas makes computing quantiles easy. This is how to get the median of a Series:

In [ ]:
bikes['atemp'].quantile(0.5)

Of course, the `quantiles` method can take a tuple as its argument. Compute the 10th, 25th, 50th, 75th, and 90th percentiles in one line of code:

In [ ]:
#ANSWER
bikes['atemp'].quantile([0.1, 0.25, 0.5, 0.75, 0.9])

### Cuts

Sometimes we want to split the sample not by the quantiles of the distribution but by the range of the data. Let's take a closer look at `atemp`:

In [ ]:
type(bikes['atemp'])

In [ ]:
bikes.sample(5)

Suppose we decide to sort these values into 4 bins of equal width, but we want to apply the resulting groups to the entire DataFrame. Basically, we need to add a row label that indicates which bin each sample belongs in. Let's call this label "atemp_level", and use the `cut` method to populate it:

In [ ]:
atemp_level = pd.cut(bikes['atemp'], bins = 4)

What is `atemp_level`?

In [ ]:
#ANSWER
print(atemp_level)

Here is a random sample of `atemp_level`:

In [ ]:
atemp_level.sample(5)

So, by default, `cut` produces labels that indicate the bin boundaries for each element in the series it was applied to. Usually, we will specify labels that are appropriate to the discretisation we are applying:

In [ ]:
atemp_level = pd.cut(bikes['atemp'], bins = 4, labels = ["cool", "mild", "warm", "hot"])
atemp_level.sample(5)

Incorporate the new `atemp_level` column into the `bikes` DataFrame and use it to count the number of "mild" `atemp` entries in `season` 2:

In [ ]:
#ANSWER
bikes['atemp_level'] = atemp_level
mild = bikes['atemp_level'] == 'mild'
print(mild.values.sum())

In [ ]:
bikes['atemp_level'].value_counts()

*Nb. The `atemp_level` variable we created is what the R language calls a "factor". Pandas has introduced a new data type called "category" that is similar to R's factors.*

# Synthetic Data

Sometimes we may want to generate test data, or we may need to initialise a series, matrix, or data frame for input to an algorithm. Numpy has several methods we can use for this.

Execute the following, then check the shape and content of each variable:

In [ ]:
# Creating arrays with initial values
a = np.zeros((3))
b = np.ones((1,3))
c = np.random.randint(1,10,(2,3,4))   # randint(low, high, size)
d = np.arange(4)
e = np.array( [[1,2,3,4], [5,6,7,8]] )

## Load Data

Load rock.csv and clean the dataset.

In [197]:
# ANSWER
df = pd.read_csv('../../DATA/rock.csv')
df.head()

,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1


In [198]:
df.describe()

,First?,Year?,PlayCount,F*G
count,2230.0,2230.000000,2230.000000,2230.000000
mean,1.0,0.741256,16.872646,15.048430
std,0.0,0.438043,25.302972,25.288366
min,1.0,0.000000,0.000000,0.000000
25%,1.0,0.000000,1.000000,0.000000
50%,1.0,1.000000,4.000000,3.000000
75%,1.0,1.000000,21.000000,18.000000
max,1.0,1.000000,142.000000,142.000000


In [199]:
df.isnull().values.sum()

577

In [200]:
df.isnull().any(axis=0)

Song Clean      False
ARTIST CLEAN    False
Release Year     True
COMBINED        False
First?          False
Year?           False
PlayCount       False
F*G             False
dtype: bool

In [201]:
df[df.isnull().any(axis=1)]

,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
10,"Baby, Please Don't Go",AC/DC,NaN,"Baby, Please Don't Go by AC/DC",1,0,1,0
13,CAN'T STOP ROCK'N'ROLL,AC/DC,NaN,CAN'T STOP ROCK'N'ROLL by AC/DC,1,0,5,0
16,Girls Got Rhythm,AC/DC,NaN,Girls Got Rhythm by AC/DC,1,0,24,0
24,Let's Get It Up,AC/DC,NaN,Let's Get It Up by AC/DC,1,0,4,0
...,...,...,...,...,...,...,...,...
2216,"I'm Bad, I'm Nationwide",ZZ Top,NaN,"I'm Bad, I'm Nationwide by ZZ Top",1,0,10,0
2218,Just Got Paid,ZZ Top,NaN,Just Got Paid by ZZ Top,1,0,2,0
2221,My Head's In Mississippi,ZZ Top,NaN,My Head's In Mississippi by ZZ Top,1,0,1,0
2222,Party On The Patio,ZZ Top,NaN,Party On The Patio by ZZ Top,1,0,14,0


In [202]:
df['Release Year'] = df['Release Year'].replace(np.nan, 0)

In [203]:
df.isnull().values.sum()

0

In [204]:
df.head(15)

,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,0,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1
5,Kryptonite,3 Doors Down,2000,Kryptonite by 3 Doors Down,1,1,13,13
6,Loser,3 Doors Down,2000,Loser by 3 Doors Down,1,1,1,1
7,When I'm Gone,3 Doors Down,2002,When I'm Gone by 3 Doors Down,1,1,6,6
8,What's Up?,4 Non Blondes,1992,What's Up? by 4 Non Blondes,1,1,3,3
9,Take On Me,a-ha,1985,Take On Me by a-ha,1,1,1,1


## Check Column Names

Check column names and clean.

In [205]:
# ANSWER

df.columns.to_list()

['Song Clean',
 'ARTIST CLEAN',
 'Release Year',
 'COMBINED',
 'First?',
 'Year?',
 'PlayCount',
 'F*G']

In [206]:
df = df.rename(columns={'ARTIST CLEAN': 'Artist Clean', 'COMBINED': 'Combined', 'F*G': 'FG'})

In [207]:
df.head()

,Song Clean,Artist Clean,Release Year,Combined,First?,Year?,PlayCount,FG
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,0,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1


## Replace Null Values With 0

Check 'release_year' column whether this column has any null values or not. Replace null values with 0.

In [208]:
# ANSWER

df['Release Year'] = df['Release Year'].replace(np.nan, 0)

## Check Datatypes of Dataset

Check datatypes of the dataset. Is there any column which should be int instead of object? Fix the column.

In [209]:
# ANSWER

df.dtypes

Song Clean      object
Artist Clean    object
Release Year    object
Combined        object
First?           int64
Year?            int64
PlayCount        int64
FG               int64
dtype: object

In [210]:
df['Song Clean'] = df['Song Clean'].astype(str)
df['Artist Clean'] = df['Artist Clean'].astype(str)
# df['Release Year'] = df['Release Year'].astype(int)
df['Combined'] = df['Combined'].astype(str)

In [211]:
df.dtypes

Song Clean      object
Artist Clean    object
Release Year    object
Combined        object
First?           int64
Year?            int64
PlayCount        int64
FG               int64
dtype: object

In [212]:
df['Release Year'] = df['Release Year'].astype(int)

ValueError: invalid literal for int() with base 10: 'SONGFACTS.COM'

In [213]:
df[df['Release Year'].str.len() > 4]

,Song Clean,Artist Clean,Release Year,Combined,First?,Year?,PlayCount,FG
1504,Bullfrog Blues,Rory Gallagher,SONGFACTS.COM,Bullfrog Blues by Rory Gallagher,1,1,1,1


In [214]:
df.loc[1504, 'Release Year'] = 0

In [215]:
df[df['Release Year'].str.len() > 4]

,Song Clean,Artist Clean,Release Year,Combined,First?,Year?,PlayCount,FG


In [216]:
df['Release Year'] = df['Release Year'].astype(int)

In [217]:
df.dtypes

Song Clean      object
Artist Clean    object
Release Year     int32
Combined        object
First?           int64
Year?            int64
PlayCount        int64
FG               int64
dtype: object

## Check Min, Max of Each Column

Is there any illogical value in any column? How can we fix that?

In [218]:
# ANSWER
df_min, df_max = (df.min(), df.max())
print(df_min, df_max)

Song Clean                     #9 Dream
Artist Clean                .38 Special
Release Year                          0
Combined        #9 Dream by John Lennon
First?                                1
Year?                                 0
PlayCount                             0
FG                                    0
dtype: object Song Clean                     Ziggy Stardust
Artist Clean                             a-ha
Release Year                             2014
Combined        Ziggy Stardust by David Bowie
First?                                      1
Year?                                       1
PlayCount                                 142
FG                                        142
dtype: object


## Write Some Functions

### Write a function that will take a row of a DataFrame and print out the song, artist, and whether or not the release date is < 1970

In [181]:
# ANSWER

def song_1970(a):
    print("Song: " + df.loc[a, 'Song Clean'])
    print("Artist: " + df.loc[a, 'Artist Clean'])
    if df.loc[a, 'Release Year'] == 0:
        print("Release date is unknown"),
    elif df.loc[a, 'Release Year'] < 1970:
        print("Release Year: before 1970")
    else:
        print("Release Year: after 1970")

In [184]:
song_1970(3)

Song: Rockin' Into the Night
Artist: .38 Special
Release Year: after 1970


### Write a function that converts a column in a DataFrame to a numeric type and otherwise replaces entries with np.nan

In [186]:
# ANSWER

def convert_to_numeric(data, column_name):
    data[column_name] = pd.to_numeric(data[column_name], errors='coerce')
    return df

### Apply this last function to your dataset

In [220]:
# ANSWER

convert_to_numeric(df, 'FG')

,Song Clean,Artist Clean,Release Year,Combined,First?,Year?,PlayCount,FG
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,0,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1
...,...,...,...,...,...,...,...,...
2225,She Loves My Automobile,ZZ Top,0,She Loves My Automobile by ZZ Top,1,0,1,0
2226,Tube Snake Boogie,ZZ Top,1981,Tube Snake Boogie by ZZ Top,1,1,32,32
2227,Tush,ZZ Top,1975,Tush by ZZ Top,1,1,109,109
2228,TV Dinners,ZZ Top,1983,TV Dinners by ZZ Top,1,1,1,1


### 'Describe' the new DataFrame

In [221]:
# ANSWER

df.describe()

,Release Year,First?,Year?,PlayCount,FG
count,2230.000000,2230.0,2230.000000,2230.000000,2230.000000
mean,1465.331390,1.0,0.741256,16.872646,15.048430
std,867.196161,0.0,0.438043,25.302972,25.288366
min,0.000000,1.0,0.000000,0.000000,0.000000
25%,0.000000,1.0,0.000000,1.000000,0.000000
50%,1973.000000,1.0,1.000000,4.000000,3.000000
75%,1981.000000,1.0,1.000000,21.000000,18.000000
max,2014.000000,1.0,1.000000,142.000000,142.000000


>




---



---



> > > > > > > > > © 2023 Institute of Data


---



---



